# Lab 4: Collecting Data from Leuschner

In [ ]:
# ssh -XY radiolab@leuschner.berkeley.edu -p 31
# password = Cud4b4ck
# cd anoising

In [2]:
import ugradio
import numpy as np
import astropy
import pyfits
from ugradio import nch
from astropy.io import fits
from ugradio import agilent
from ugradio import leo
from ugradio import leusch
import Leuschner

ImportError: No module named Leuschner

## Telescope Controls

In [ ]:
tele = ugradio.leusch.LeuschTelescope() # interface for controlling the Leuschner Telescope
tele.get_pointing()  # returns the current (alt,az) of the dish in degrees
tele.point(alt,az) # moves the telescope to a specified alt, az in degrees
tele.main() # moves the telescope to maintenance position

## Noise

In [ ]:
LeuschNoise = ugradio.leusch.LeuschNoise() # interface to the noise diode
LeuschNoise.on()
LeuschNoise.off()

## Local Oscillator (LO)

In [ ]:
lo = ugradio.agilent.SynthDirect() # interface to the LO
lo.get_frequency()
lo.get_amplitude()
lo.set_frequency(633, 'MHz')
lo.set_amplitude(10, 'dBm')

synth = ugradio.agilent.SynthClient(host='127.0.0.1') # idk what this does

## Spectrometer Controls

In [2]:
spec = leuschner.Spectrometer('10.0.1.2') # interface to the spectrometer at the specified IP address
spec.check_connected() # checks that you are connected and ready to take data
spec.int_time() # returns how long each spectrum is integrated for
spec.read_spec('name.fits', N, (ra, dec), 'eq') # reads N spectra and stores output in a fits file, writing ra/dec coordinates into the header of the file. Note that this does NOT point the telescope.
# e.g. spec.read_spec('data_apr20.fits', 15,(50, 60),'eq')

## For Loops to Point Telescope and Collect Data

Note: replace 1930 with appropriate time

tele = ugradio.leusch.LeuschTelescope()

lo = ugradio.agilent.SynthDirect()
lo.set_frequency(710, 'MHz')
lo.set_amplitude(10, 'dBm')

spec = leuschner.Spectrometer('10.0.1.2')

print('system ready')

LeuschNoise = ugradio.leusch.LeuschNoise()
LeuschNoise.on()

l = 115
b = 31

galactic = SkyCoord(l, b, frame='galactic', unit='deg')
icrs = galactic.transform_to('icrs')
ra = icrs.ra.degree
dec = icrs.dec.degree
lat = leo.lat
lon = leo.lon
alt = leo.alt
jd = ugradio.timing.julian_date()
t = astropy.time.Time(jd,format='jd')
l = astropy.coordinates.EarthLocation(lat=lat*u.deg,lon=lon*u.deg,height = alt*u.m)
f = astropy.coordinates.AltAz(obstime=t,location=l)
c = astropy.coordinates.SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2019')
altaz = c.transform_to(f)
alt = altaz.alt.deg
az = altaz.az.deg
tele.point(alt, az)
spec.read_spec('noise_on_1930.fits', 15, (ra, dec), 'eq')

print('noise on observation complete')

LeuschNoise.off()

spec.read_spec('noise_off_1930.fits', 15, (ra, dec), 'eq')

print('noise off observation complete')


l = np.arange(l1, l2, 2)
b = np.arange(b1, b2, 2)

ra_dec = []

for i in range(len(l)):
    for j in range(len(b)):
        galactic = SkyCoord(l[i], b[j], frame='galactic', unit='deg')
        icrs = galactic.transform_to('icrs')
        ra = icrs.ra.degree
        dec = icrs.dec.degree
        jd = ugradio.timing.julian_date()
        lat = leo.lat
        lon = leo.lon
        alt = leo.alt
        t = astropy.time.Time(jd,format='jd')
        l = astropy.coordinates.EarthLocation(lat=lat*u.deg,lon=lon*u.deg,height = alt*u.m)
        f = astropy.coordinates.AltAz(obstime=t,location=l)
        c = astropy.coordinates.SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2019')
        altaz = c.transform_to(f)
        alt = altaz.alt.deg
        az = altaz.az.deg
        tele.point(alt, az)
        spec.read_spec('data_date_'+str(ra)+'_'+str(dec)+'.fits', 15, (ra, dec), 'eq')

tele = ugradio.leusch.LeuschTelescope()

lo = ugradio.agilent.SynthDirect()
lo.set_frequency(710, 'MHz')
lo.set_amplitude(10, 'dBm')

spec = leuschner.Spectrometer('10.0.1.2')

print('system ready')

LeuschNoise = ugradio.leusch.LeuschNoise()
LeuschNoise.on()

l = 115
b = 31

galactic = SkyCoord(l, b, frame='galactic', unit='deg')
icrs = galactic.transform_to('icrs')
ra = icrs.ra.degree
dec = icrs.dec.degree
lat = leo.lat
lon = leo.lon
alt = leo.alt
jd = ugradio.timing.julian_date()
t = astropy.time.Time(jd,format='jd')
l = astropy.coordinates.EarthLocation(lat=lat*u.deg,lon=lon*u.deg,height = alt*u.m)
f = astropy.coordinates.AltAz(obstime=t,location=l)
c = astropy.coordinates.SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2019')
altaz = c.transform_to(f)
alt = altaz.alt.deg
az = altaz.az.deg
tele.point(alt, az)
spec.read_spec('noise_on_1930.fits', 15, (ra, dec), 'eq')

print('noise on observation complete')

LeuschNoise.off()

spec.read_spec('noise_off_1930.fits', 15, (ra, dec), 'eq')

print('noise off observation complete')

gal = [[115, 31], [115, 33], [117, 31], [117, 33], [119, 31], [119, 33]]

for i in range(len(gal)):
    l = gal[i][0]
    b = gal[i][1]
    galactic = SkyCoord(gal[i][0], gal[i][1], frame='galactic', unit='deg')
    icrs = galactic.transform_to('icrs')
    ra = icrs.ra.degree
    dec = icrs.dec.degree
    jd = ugradio.timing.julian_date()
    lat = leo.lat
    lon = leo.lon
    alt = leo.alt
    t = astropy.time.Time(jd,format='jd')
    l = astropy.coordinates.EarthLocation(lat=lat*u.deg,lon=lon*u.deg,height = alt*u.m)
    f = astropy.coordinates.AltAz(obstime=t,location=l)
    c = astropy.coordinates.SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2019')
    altaz = c.transform_to(f)
    alt = altaz.alt.deg
    az = altaz.az.deg
    tele.point(alt, az)
    spec.read_spec('data_new_1930'+str(ra)+'_'+str(dec)+'.fits', 15, (ra, dec), 'eq')

Redoing Observations:

1:00 -- [119, 25]

3:00 -- [115, 19], [115, 21], [117, 19], [117, 21], [117, 23], [117, 25], [119, 19], [119, 21], [119, 23], [121, 23]

6:00 -- [121, 21], [123, 21], [123, 23], [125, 21], [125, 23], [127, 17], [127, 19], [127, 21], [127, 23], [129, 15], [129, 17], [129, 19], [129, 21], [129, 23]

9:00 -- [127, 25]

10:00 -- l (105 to 114), b (15 to 34)

11:00 -- [129, 25], [129, 27], [129, 29]

12:00 -- l (131 to 160), b (15 to 34)

13:00 -- [127, 29]

16:00 -- [121, 33], [123, 33], [125, 31], [125, 33], [127, 31], [127, 33], [129, 31], [129, 33]

16:30 --  l (105 to 160), b (35 to 50)

19:30 -- [115, 31], [115, 33], [117, 31], [117, 33], [119, 31], [119, 33] -- done

19:55 -- [121, 31] -- done

21:30 -- [119, 29] -- not in range?

22:00 -- [115, 23], [115, 25], [115, 27], [115, 29], [117, 27], [117, 29]